# Importing Libraries

In [1]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt

import cv2
import pandas as pd
import numpy as np
import mahotas

from PIL import Image,ImageOps

from skimage.feature import hog
from skimage.color import rgb2grey

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

# Image and Data Preprocessing and Feature Extraction

## Feature Extraction Functions

In [2]:
def fd_hu_moments(image):
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def fd_haralick(image): 
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(image).mean(axis=0)
    return haralick

def create_features(img):
    # flatten three channel color image
    #color_features = img.flatten()
    # convert image to greyscale
    grey_image = rgb2grey(img)
    # get HOG features from greyscale image
    hog_features,hog_imgs = hog(grey_image, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2),visualize=True)  #got total 324 features,after checking I wrote this.
    # combine all features into a single array
    hu_features = fd_hu_moments(img)  #Total 7 features
    haralick_features = fd_haralick(img)  #Total 13 features
    #print('no. of features => hog=',len(hog_features),', hu=',len(hu_features),', haralick=',len(haralick_features))
    flat_features = np.hstack([hog_features,hu_features,haralick_features])
    return flat_features

In [3]:
def getImage(path):
    img = Image.open(path)
    img = ImageOps.fit(img,(64,64),Image.ANTIALIAS)
    img = np.array(img)
    return img

def getDF(path): #get DataFrame
    features_list=[]
    for file in os.listdir(path):
        path1 = os.path.join(path+file)
        img = getImage(path1)
        fr = create_features(img)
        features_list.append(fr)
    feature_matrix = np.array(features_list)
    return feature_matrix

### Preparing Training Set

In [4]:
train1 = getDF('.\\train\\NORMAL\\')

In [10]:
train1

array([[ 0.13829613,  0.        ,  0.        , ...,  5.17048539,
        -0.38483317,  0.99837777],
       [ 0.2520088 ,  0.30773162,  0.30773162, ...,  5.35489435,
        -0.36338669,  0.99783129],
       [ 0.3244928 ,  0.03073737,  0.        , ...,  5.09888993,
        -0.35587406,  0.99720542],
       ...,
       [ 0.39171518,  0.19374719,  0.        , ...,  4.69333821,
        -0.34790142,  0.99617359],
       [ 0.12677086,  0.        ,  0.13111985, ...,  4.82680968,
        -0.35097743,  0.99651501],
       [ 0.33712332,  0.        ,  0.        , ...,  4.87480457,
        -0.38654834,  0.99814414]])

In [8]:
train1.shape

(1341, 8120)

In [11]:
pca = PCA(n_components=100)
train1 = pca.fit_transform(train1)

In [12]:
train1.shape

(1341, 100)

In [13]:
ss = StandardScaler()
train1 = ss.fit_transform(train1)

In [14]:
label1 = np.array(['normal']*1341).reshape(-1,1) #1341 is no. of rows in train1
train1 = np.concatenate((train1,label1),axis=1)

In [15]:
train1

array([['0.6864043909191616', '-1.1578634753077777',
        '-0.14978184392678884', ..., '-0.266531406013027',
        '-0.09254903651618077', 'normal'],
       ['0.01234082264659507', '0.8260274065765886',
        '1.4452979015586724', ..., '0.5513439925735877',
        '-0.4151086687538014', 'normal'],
       ['-0.38717981596379225', '-0.48466938275323734',
        '-0.5558592264756236', ..., '-0.543353147694312',
        '-1.3317208106625964', 'normal'],
       ...,
       ['-1.0734563067236438', '-1.7159640591639018',
        '1.7986716440447694', ..., '2.1460549701265577',
        '-0.556784473973163', 'normal'],
       ['-1.01467170397635', '-1.0817555352470756',
        '-0.5945200891874015', ..., '0.2768146149408373',
        '0.7258556862560157', 'normal'],
       ['0.20280017928417038', '-1.9517575114412793',
        '2.605164445736288', ..., '1.0906475586952997',
        '-0.7781712865596657', 'normal']], dtype='<U32')

In [16]:
# Doing same operations to get train2(Pneumonia part)
train2 = getDF('.\\train\\PNEUMONIA\\')
pca = PCA(n_components=100)
train2 = pca.fit_transform(train2)
ss = StandardScaler()
train2 = ss.fit_transform(train2)

In [17]:
train2.shape

(3592, 100)

In [18]:
label2 = np.array(['pneumonia']*3592).reshape(-1,1) #3592 is no. of rows in train2
train2 = np.concatenate((train2,label2),axis=1)

In [19]:
train2.shape

(3592, 101)

In [20]:
train_set = np.concatenate((train1,train2),axis=0)

In [21]:
train_set

array([['0.6864043909191616', '-1.1578634753077777',
        '-0.14978184392678884', ..., '-0.266531406013027',
        '-0.09254903651618077', 'normal'],
       ['0.01234082264659507', '0.8260274065765886',
        '1.4452979015586724', ..., '0.5513439925735877',
        '-0.4151086687538014', 'normal'],
       ['-0.38717981596379225', '-0.48466938275323734',
        '-0.5558592264756236', ..., '-0.543353147694312',
        '-1.3317208106625964', 'normal'],
       ...,
       ['0.26367279199178006', '-1.0559742194290351',
        '-0.18049301706116777', ..., '-1.380185542194111',
        '-0.8371160865995119', 'pneumonia'],
       ['-0.9978129912932912', '0.1965075016450941',
        '-0.8198437032138226', ..., '-0.03522930142488456',
        '0.00882506744500263', 'pneumonia'],
       ['-0.5403224282990343', '-0.6383764105905336',
        '0.4344641212266098', ..., '-0.3596217694403383',
        '-0.12446372531956615', 'pneumonia']], dtype='<U32')

In [22]:
train_set.shape

(4933, 101)

In [23]:
np.random.shuffle(train_set)

In [22]:
train_set

array([['1.2708176814340306', '-1.5906594389338347',
        '1.4577351540280412', ..., '-0.2302765842761266',
        '-0.5913948086706202', 'pneumonia'],
       ['-0.15949459150556328', '0.09522725426458803',
        '-0.7347119625501648', ..., '-0.7486410055835269',
        '0.4263836016780033', 'pneumonia'],
       ['0.6923839973054228', '0.49389224325601266', '2.166188833952168',
        ..., '-2.5904956700061827', '-0.44881524487635954', 'pneumonia'],
       ...,
       ['1.9096313175423167', '0.09336692529687783',
        '0.7537674156873273', ..., '-0.6585973080721648',
        '0.5887410140472499', 'pneumonia'],
       ['-1.9961488242217835', '0.20529015420204283',
        '-0.8913185825217947', ..., '-0.3301031560862101',
        '1.8805190482650114', 'pneumonia'],
       ['0.13956210027994295', '-0.9085046002449325',
        '-0.07117935667350593', ..., '1.6133258891914415',
        '1.7342866553798013', 'pneumonia']], dtype='<U32')

## Preparing test set

In [24]:
test1 = getDF('.\\test\\NORMAL\\')
pca = PCA(n_components=100)
test1 = pca.fit_transform(test1)
ss = StandardScaler()
test1 = ss.fit_transform(test1)

In [25]:
test1.shape

(234, 100)

In [26]:
label1 = np.array(['normal']*234).reshape(-1,1) #234 is no. of rows in test1
test1 = np.concatenate((test1,label1),axis=1)

In [27]:
test1

array([['0.2435974505168652', '-0.20022577281389617',
        '-0.0714450980205829', ..., '0.2715431284374765',
        '1.4142648225207914', 'normal'],
       ['-0.3189901459820519', '0.17915871217475202',
        '-1.3390605474782116', ..., '-0.11387714473892618',
        '-0.10840600806874298', 'normal'],
       ['0.548359998105811', '-0.7729182156077122',
        '-0.1565039326723877', ..., '-0.9888292880158444',
        '0.6602845993509231', 'normal'],
       ...,
       ['-0.5811702389675427', '0.6214823875948488',
        '1.8871507910294323', ..., '0.9571549171394201',
        '-1.0552814941348532', 'normal'],
       ['0.03400204407887488', '-0.48859107325734275',
        '-0.25059911934538437', ..., '0.9115307648919669',
        '0.616068537271108', 'normal'],
       ['-0.17690917421935723', '-0.9404727800001073',
        '-0.022728557846579622', ..., '0.48535151492965567',
        '-0.8314869392686487', 'normal']], dtype='<U32')

In [28]:
test2 = getDF('.\\test\\PNEUMONIA\\')
pca = PCA(n_components=100)
test2 = pca.fit_transform(test2)
ss = StandardScaler()
test2 = ss.fit_transform(test2)

In [29]:
test2.shape

(390, 100)

In [30]:
label2 = np.array(['pneumonia']*390).reshape(-1,1) #390 is no. of rows in test2
test2 = np.concatenate((test2,label2),axis=1)

In [31]:
test2.shape

(390, 101)

In [32]:
test_set = np.concatenate((test1,test2),axis=0)

In [33]:
test_set.shape

(624, 101)

In [34]:
test_set

array([['0.2435974505168652', '-0.20022577281389617',
        '-0.0714450980205829', ..., '0.2715431284374765',
        '1.4142648225207914', 'normal'],
       ['-0.3189901459820519', '0.17915871217475202',
        '-1.3390605474782116', ..., '-0.11387714473892618',
        '-0.10840600806874298', 'normal'],
       ['0.548359998105811', '-0.7729182156077122',
        '-0.1565039326723877', ..., '-0.9888292880158444',
        '0.6602845993509231', 'normal'],
       ...,
       ['-0.36355557355225915', '0.7670313762901496',
        '-1.243197654458368', ..., '-1.696556189162131',
        '1.072775091836771', 'pneumonia'],
       ['0.36613780798917467', '0.28965492998255554',
        '-0.09134331678693175', ..., '-0.46568585432133086',
        '0.8260306197907766', 'pneumonia'],
       ['1.5024863252450076', '0.5041729270324699',
        '-0.6276581888932901', ..., '0.5750047743286817',
        '-0.3523680732404671', 'pneumonia']], dtype='<U32')

In [35]:
np.random.shuffle(test_set)  #so as to mix data

In [36]:
test_set

array([['-0.472328445619833', '1.0856749104255496',
        '-0.7498223394349696', ..., '-0.18517366587103312',
        '-0.04209246187223929', 'normal'],
       ['-1.6112131578610644', '0.13864809854227939',
        '-0.30914738059180874', ..., '1.0343751798653746',
        '0.07364042899342974', 'normal'],
       ['-1.3952673776523947', '0.49855169717188313',
        '-0.49942465842019046', ..., '0.9189371578179882',
        '-0.8814727071071546', 'pneumonia'],
       ...,
       ['0.5238222497771975', '-1.5117627302426342',
        '0.4337867584050859', ..., '0.6660620677972664',
        '1.675814387586313', 'pneumonia'],
       ['-0.833359176399394', '0.5555129678149998',
        '-1.4567653884719358', ..., '-0.11672203664793739',
        '0.7853140579003111', 'pneumonia'],
       ['1.3380108104140156', '-0.57214533414723', '-1.0827398370118677',
        ..., '0.1924176694792265', '-0.5225341168863505', 'pneumonia']],
      dtype='<U32')

### Now Our train_set and test_set is ready 

### Creating DataFrames from Numpy Arrays and then saving to CSV files

In [37]:
train = pd.DataFrame(train_set)
test = pd.DataFrame(test_set)

In [38]:
train.to_csv('train_set_new(1).csv')
test.to_csv('test_set_new(1).csv')

In [39]:
train

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.1572669686353674,-0.5751435673047,0.12668787978384385,0.6768619115638205,0.12646136564845298,-1.9198492690708728,1.7324391892693887,0.6165822626790302,0.3293250216180348,-1.1493721861709472,...,-0.9933243349219238,-1.6304201463070906,0.023476119328459357,-1.5398981130055838,0.8871276760053102,-0.5390077421300565,0.9770791308098298,0.7377718992091017,-0.48151133301988425,pneumonia
1,-0.8357224368221275,-1.7313486451456674,-0.2832128892433425,-0.3267970267264208,2.225977714144977,0.519542913738108,-0.05967076909497569,-0.9518324090548493,1.0511325714207704,-1.5423913191516938,...,1.4286403694254382,0.21492445281762396,0.2367266548234994,0.7356566412318781,0.7933243514723572,-0.9477045363246078,-0.043213843456927394,-0.06089144638060754,0.4523886393127524,normal
2,-0.31020029495836293,0.7855158527897361,-1.2895001419823624,-1.0894517782157338,-0.12850648608614182,-0.19491822694475236,1.7179805385261233,-0.7484331389259488,-1.279297905819819,1.7028705588642876,...,-1.034225142322296,-0.20892849239310327,0.4664602985182261,-0.7332807654894302,0.29346437381590657,0.01590860525721144,0.12991518808013874,0.6082296408769199,1.0033015449687213,normal
3,0.3867202734341585,-0.5595308940567283,-0.5285513844520184,0.9997883417388166,2.0793171054372417,-0.9376887471568977,-0.492586425127101,-0.47758461515481443,0.9003613071186041,1.6670533788032431,...,-0.006582665608985058,-0.39921817601812565,-1.2021249547659822,2.080307440363077,-0.4231465421115514,1.6474135122194047,1.1499268719934541,-2.4289439845639307,1.754038529665183,pneumonia
4,-1.0129177077795697,0.23905031743388666,-1.0146668934900887,0.7433396123270157,-0.6166523282832391,0.09682027483135928,0.44368493106185103,2.6755468625279915,0.5250624250062926,0.8339258731962852,...,1.7671982480800932,-0.7832794412544125,0.4076333166170647,-0.4636022810525782,0.6093340070964288,1.1197179326396443,0.6625438408035123,-1.0780701074124028,0.393151139887864,pneumonia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4928,-0.6220917522521094,-0.3290027006663462,3.225325524561807,-0.14152420404212124,0.659888585080716,-1.817976784245516,-0.276273706745958,0.7860533259120324,-0.20727182920933399,-0.5263355834658527,...,0.47110169040060074,-0.6871897454001419,0.30430400270268193,-1.0858956665638266,-0.5622427516054236,1.7648424812985615,-0.48986291253953945,1.1106223195289977,0.5868364879079779,pneumonia
4929,-1.0515478795743878,1.1733035817456687,-0.00296470764511297,-0.3172892482184532,-0.3172478584225614,1.9851749591824865,1.0222493311619267,0.49245903876034636,1.2231284847888888,-0.6294687950854969,...,-0.7308752306879673,0.5511832205189225,-0.33428357772397715,0.2693068778989378,-0.3352565844344862,-0.6821032711567999,-0.8909170497840734,-0.3422306181909494,-0.5723155439183917,normal
4930,-0.21058937463242725,-0.6801585219212779,2.0241955022781726,-0.03779874840838618,0.1351427209762704,-0.9519159113292575,-0.6023554507203002,2.6588437535775373,1.0945050189965162,0.11566367466804894,...,-0.9504610834408538,-0.9464277965994826,-1.597649504451558,0.5546779458333815,0.13204839064084106,-2.051182513989279,-0.5807657171233044,1.3655499720559499,-0.1492962047878799,pneumonia
4931,2.300920081949459,1.6968655542823687,-0.036064676922291254,-1.1542843848544975,0.6399207024915792,0.8567945581869038,-1.3109279782217267,-0.7030273344612862,0.2697635038391198,0.554876353878161,...,-0.5496357442514755,-2.4014812331062245,0.46396197067402534,-1.656399901064892,0.20800802659430218,-0.9440476895859031,0.8887237953263709,-0.1927811481367094,0.3072388442843738,pneumonia


In [40]:
test

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,-0.472328445619833,1.0856749104255496,-0.7498223394349696,-0.28546741290400235,-0.6605396680202078,1.1559979843599213,1.4794016430836576,2.15320274713804,-0.7364165612510152,-0.5757419568633225,...,-0.6281149271212726,2.7650397892172216,0.3881212100210379,0.01781955262311996,0.5642658508498739,3.5880236741270175,1.2541843479752528,-0.18517366587103312,-0.04209246187223929,normal
1,-1.6112131578610644,0.13864809854227939,-0.30914738059180874,0.43426502539878725,1.2728246102712868,0.9180796591736442,-0.86023493616095,0.47094557777077317,1.993506611985877,0.19778872771441705,...,0.6674312350822625,-0.2804143479302836,0.7180706435319085,-0.7328752632336083,0.0021227643670673683,-0.20964886497054908,-0.5699398770960721,1.0343751798653746,0.07364042899342974,normal
2,-1.3952673776523947,0.49855169717188313,-0.49942465842019046,1.3210796058087326,-0.04665892869271664,0.6030964919311028,-0.7527919912364244,1.658665566536212,-0.3478204833248322,-1.2543084590726163,...,0.07801965003516405,-1.768159313815759,0.3829998016719104,-0.5615095463375139,2.1624613362951077,2.4586357072554192,-1.7344967698418294,0.9189371578179882,-0.8814727071071546,pneumonia
3,-0.18281017537755473,-0.46649077825680835,0.3339162358458726,0.15199000596105314,-1.2880838073385181,0.6675363757445141,-0.6142379517779144,1.0653365769036247,0.09336352322026643,1.3063378605220788,...,-0.7279842150034382,-0.18025492535409474,-0.04303890090843188,0.16946606578800066,0.7702358710675781,-0.26762330165167636,-0.9242217738528867,0.7192769695043251,1.59010302325686,normal
4,0.07626927209646203,-0.4953973127813742,-0.6319941393038025,-1.4987221595310685,0.9311405347800872,0.5298888658260776,-0.7853116860066612,1.255342026495359,-1.2564783075337138,0.9975266797921579,...,1.3734011758601452,0.1317821116051737,0.8889059770333151,-2.067775985076586,-2.914500138460401,-0.9910649633738259,0.757488617804167,-0.8106010455778169,0.7207733553175807,pneumonia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,0.03639715930746186,0.7823066880840358,-0.3113275790278836,2.2756542749549737,0.7431507012367299,-0.1699072411135659,0.6418198367471165,-0.39331763748565574,0.10873009676236084,1.3287015976528926,...,-1.012573856380713,0.6418994309697126,0.38822090899349926,-0.8046948758177994,0.3117366426412593,-1.939007269836996,-1.6089598684835507,2.2801905617909135,0.985554938023258,pneumonia
620,-1.2858367272031985,-0.09786916758070803,-1.6542770047995572,2.41535493182398,0.0011433508334466466,0.7233494644296105,-2.388542055009293,0.6366378679989734,0.49111733597736323,-0.9752327172568886,...,-1.6734154849196288,0.9993842723151426,0.4628880463954262,0.6673738351914519,-1.094061669753242,0.8643626712542201,1.6570935011029104,1.2699034795177928,-1.383064545357036,pneumonia
621,0.5238222497771975,-1.5117627302426342,0.4337867584050859,0.21110394324079676,-0.08968876033918047,-0.5221752653369808,1.7112085238165038,0.4594712405765985,-1.8703578220525747,-0.39937879128606046,...,0.5136496778685096,-0.053010218711275515,0.355692370643106,-0.23602347463785286,-0.5041837061896773,-0.1438119596063305,-0.518827237889777,0.6660620677972664,1.675814387586313,pneumonia
622,-0.833359176399394,0.5555129678149998,-1.4567653884719358,-0.7363407469810624,-0.22629740308460525,2.2644323430572717,-0.9081605354416695,0.42527642541770544,-0.45845550983165495,1.1637344096190032,...,-2.4349923962970945,-1.3421895111788302,0.9234631128214166,0.6233034169342374,-0.5190733716315702,0.20608546114074117,0.3777488306281593,-0.11672203664793739,0.7853140579003111,pneumonia


In [41]:
X_train = train.iloc[:, :-1].values
Y_train = train.iloc[:,-1].values
X_test = test.iloc[:, :-1].values
Y_test = test.iloc[:,-1].values

# Training a Kernel SVM model

In [42]:
classifier = SVC(kernel = 'rbf')
classifier.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Making prediction on test set

In [46]:
Y_pred = classifier.predict(X_test)
Y_predTrain = classifier.predict(X_train)

## Evaluating Results

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_train, Y_predTrain)  #Training Accuaracy

0.7281573079262113

In [48]:
accuracy_score(Y_test, Y_pred)  #Testing Accuracy

0.625

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[  0, 234],
       [  0, 390]], dtype=int64)

In [50]:
confusion_matrix(Y_train, Y_predTrain)

array([[   0, 1341],
       [   0, 3592]], dtype=int64)

# Training a linear SVM model

In [51]:
classifier = SVC(kernel = 'linear')
classifier.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Making prediction on test set

In [52]:
Y_pred = classifier.predict(X_test)
Y_predTrain = classifier.predict(X_train)

## Evaluating Results

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_train, Y_predTrain)  #Training Accuaracy

0.7281573079262113

In [54]:
accuracy_score(Y_test, Y_pred)  #Testing Accuracy

0.625

In [55]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[  0, 234],
       [  0, 390]], dtype=int64)

In [56]:
confusion_matrix(Y_train, Y_predTrain)

array([[   0, 1341],
       [   0, 3592]], dtype=int64)

# Training a KNN model

In [57]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

## Making prediction on test set

In [58]:
Y_pred = classifier.predict(X_test)
Y_predTrain = classifier.predict(X_train)

## Evaluating Results

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_train, Y_predTrain)  #Training Accuaracy

0.9172917088992499

In [60]:
accuracy_score(Y_test, Y_pred)  #Testing Accuracy

0.5801282051282052

In [61]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[ 35, 199],
       [ 63, 327]], dtype=int64)

In [62]:
confusion_matrix(Y_train, Y_predTrain)

array([[1006,  335],
       [  73, 3519]], dtype=int64)